### Problem Statement   
*   How to better stock the items they sell

### Performance Measure
The evaluation metric for this Project is AUC (area under the curve). The AUC (Area Under Curve) is the area enclosed by the ROC curve. A perfect classifier has AUC = 1 and a completely random classifier has AUC = 0.5. Hence, this classification project will be evaluated as a good predictive if the AUC will be closer to 1.

### Data description
sku -sku code   
national_inv- Current inventory level of component  
lead_time -Transit time     
in_transit_qtry - Quantity in transit   
forecast_x_month - Forecast sales for the net 3, 6, 9 months    
sales_x_month - Sales quantity for the prior 1, 3, 6, 9 months      
min_bank - Minimum recommended amount in stock      
potential_issue - Indictor variable noting potential issue with item    
pieces_past_due - Parts overdue from source     
perf_x_months_avg - Source performance in the last 6 and 12 months      
local_bo_qty - Amount of stock orders overdue       
X17-X22 - General Risk Flags    
went_on_back_order - Product went on backorder      
Validation - indicator variable for training (0), validation (1), and test set (2)

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set(style="darkgrid", font_scale=1.2)

In [2]:
data = pd.read_csv("/home/miki/Desktop/Docker/iNeuron/backorder-prediction/dataset/Kaggle_Training_Dataset_v2.csv")

/tmp/ipykernel_30439/791597961.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/home/miki/Desktop/Docker/iNeuron/backorder-prediction/dataset/Kaggle_Training_Dataset_v2.csv")
